In [ ]:
# ! pip install -U mlem scikit-learn fastapi uvicorn pandas numpy docker

## Training model

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
data, y = load_iris(return_X_y=True, as_frame=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
        n_jobs=2,
        random_state=42,
)
rf.fit(data, y)

## Saving model

Let's save the model using MLEM

In [ ]:
from mlem.api import save

In [ ]:
model = save(
        rf,
        "rf",
        sample_data=data,
        description="Random Forest Classifier",
)

Let's see what we have now

In [ ]:
! ls -la .

In [ ]:
! cat rf.mlem

This yaml is just serialized `model` object

In [ ]:
model.dict()

And `rf` is just a pickle file

In [ ]:
import pickle

with open("rf", "rb") as f:
    rf2 = pickle.load(f)

rf2

`artifacts` field contains info about files. It can be used to open them

In [ ]:
with model.artifacts["data"].open() as f:
    rf3 = pickle.load(f)

rf3

`requirements` has python packages with versions pinned

In [ ]:
model.requirements.to_pip()

`model_type` is a wrapper for sklearn model. It contains original `rf` object, IO implementation and model methods 

In [ ]:
model.model_type

In [ ]:
model.model_type.dict()

In [ ]:
model.model_type.model

In [ ]:
model.model_type.io

In [ ]:
model.model_type.methods["predict"]

In [ ]:
model.model_type.methods["predict"].args[0].type_.columns

In [ ]:
model.model_type.methods["predict"].returns

## Saving data

Data also can be saved with MLEM. Later it can be used to score new model versions, for example.
API is exactly the same

In [ ]:
dataset = save(
        data,
        "iris.csv",
        description="Iris Dataset",
)

In [ ]:
dataset.dict()

In [ ]:
dataset.requirements.to_pip()

Instead of `model_type` data has `data_type` and `reader` 

In [ ]:
dataset.reader.dict()

In [ ]:
dataset.data

pandas DataFrames are saved as csv by default

In [ ]:
import pandas as pd

data2 = pd.read_csv("iris.csv")

In [ ]:
(data2 == data).all()

## MLEM Project

You can organize objects you saved with MLEM Project.

Let's clean our workspace and init MLEM Project

In [ ]:
! rm -rf rf* data.csv*

In [ ]:
! mlem init

MLEM Project actually is just a .mlem dir. For now it only holds empty config file

In [ ]:
! ls -la .mlem

Now we can change configuration. Lets set default pandas format to parquet and re-save all objects

In [ ]:
! mlem config set pandas.default_format parquet

In [ ]:
save(
        rf,
        "rf",
        sample_data=data,
        description="Random Forest Classifier",
);
save(
        data,
        "iris",
        description="Iris Dataset",
);

If you are working with MLEM Project, all the objects will be saved into .mlem directory by default

In [ ]:
! ls -la .mlem/data

In [ ]:
pd.read_parquet(".mlem/data/iris")

## MLEM CLI & API

MLEM has a bunch of commands. Most of the API commands have their CLI counterpart

In [ ]:
! mlem -h

In [ ]:
! mlem ls

In [ ]:
from mlem.api import __all__

__all__

## Loading

To load any object saved with MLEM, you should use `load` API method. 

It will return the exact same object you saved

Also there is `load_meta` that will return `MlemObject` instance with metadata (`MlemModel` or `MlemData`)

In [ ]:
from mlem.api import load, load_meta

load("rf")

Note that you don't have to specify `.mlem/model/` if you are working in MLEM Project

You also can load remote objects from cloud and github. You can even specify commit/tag/branch

Let's load a model from our Get Started repository (it should be exactly the same since it used the same training data)

In [ ]:
remote_rf = load_meta("rf", project="https://github.com/iterative/example-mlem-get-started", rev="simple")

`loc` field has info about original object location

In [ ]:
remote_rf.loc

In [ ]:
remote_rf.dict()

In [ ]:
To get original object from `MlemObject` you need to load it first and then it will be accessible

In [ ]:
remote_rf.load_value()
remote_rf.get_value()

You can also call original model methods on `MlemModel` instance 

In [ ]:
remote_rf.predict(data)

In [ ]:
rf.predict(data)

## Applying

If you just want to apply your model to your data, you can do it right from CLI


In [ ]:
! mlem apply rf iris --json

It works with remote objects too

In [ ]:
! mlem apply rf --project https://github.com/iterative/example-mlem-get-started --rev simple iris --json

## Serving

Another way to utilize the model is to wrap it with http server.

MLEM can do it with a single API/CLI call. We will use CLI because we cannot run server right from notebook

In [ ]:
from mlem.api import serve

In [ ]:
! mlem serve --help

Let's check available fields for fastapi

In [ ]:
! mlem types server fastapi

In [ ]:
! mlem serve rf fastapi -c port=80

## Builing model packages

You can also build packages from models with MLEM. 

Think of this as "exporting" to another format

Let's check available implementations

In [ ]:
from mlem.api import build

In [ ]:
! mlem types builder

Let's build a docker diractory (we can do docker image, but that might take some time)

In [ ]:
! mlem types builder docker_dir

In [ ]:
build("docker_dir", "rf", server="fastapi", target="build/docker")

Let's see what it did

In [ ]:
! ls -la build/docker/

In [ ]:
! cat build/docker/requirements.txt

In [ ]:
! cat build/docker/Dockerfile

In [ ]:
! cat build/docker/run.sh

We can also build python package

In [ ]:
! mlem types builder whl

In [ ]:
build("whl", remote_rf, package_name="my_rf", target="build/whl")

In [ ]:
! ls -la build/whl/

Let's install it and try out

In [ ]:
! pip install build/whl/my_rf-0.0.0-py3-none-any.whl --no-deps

In [ ]:
import my_rf

In [ ]:
my_rf.predict(data)

## Cloud deployment

Final option is to deploy your model to some cloud. 

This is still experimental and for now we support Heroku deployment

To do it, we need to declare target environment for deployment

In [ ]:
! mlem declare env heroku staging

In [ ]:
! mlem types env heroku

I didn't provide api_key because I did `heroku login`. 
If you did not, you should provide heroku api_key via env or via configuration

In [ ]:
from mlem.api import deploy

In [ ]:
! mlem types deployment heroku

In [ ]:
deploy("service", "rf", "staging", app_name="mlem-tutorial-notebook")

In [ ]:
Deployment metadata can be loaded just as any other MlemObject

In [ ]:
service = load_meta("service")

In [ ]:
You can use it to create client to make requests just like the model is local

In [ ]:
client = service.state.get_client()

In [ ]:
client.predict(data)

## Cleanup

In [ ]:
stop;  # stop "Run All Cells"

In [ ]:
! heroku apps:destroy mlem-tutorial-notebook --confirm mlem-tutorial-notebook

In [ ]:
! rm -rf .mlem iris* rf* build/

In [ ]:
! ls -la

In [ ]:
! pip uninstall my_rf